### Imports & Setting

In [206]:
## library imports
library(forecast)
library(fUnitRoots)
library(portes)
library(astsa)
library(sarima)

## for dataset
library(fma)
library(expsmooth)

## setting
options(repr.plot.width = 15, repr.plot.height = 8)
setwd("~/TS2024/data/Time Series Data/")

### 1번 문제

In [ ]:
##----------(6) acf/pacf plot----------
data1 = list(acf = as.numeric(strsplit("-0.48 -0.04 -0.06 0.14 -0.22 0.19 -0.10 -0.02 0.09 -0.03 -0.12 0.09 0.03", " ")[[1]]),
             pacf = as.numeric(strsplit("-0.48 -0.24 -0.21 0.01 -0.20 -0.01 -0.05 -0.15 -0.04 -0.06 -0.04 0.02 0.06", " ")[[1]]))
data2 = list(acf = as.numeric(strsplit("0.59 0.44 0.33 0.23 0.24 0.16 0.05 0.01 -0.03 -0.11 -0.08 -0.07 0.01", " ")[[1]]),
             pacf = as.numeric(strsplit("0.59 0.13 0.04 -0.02 0.12 -0.05 -0.12 -0.03 -0.01 -0.13 0.06 0.04 0.13", " ")[[1]]))
data3 = list(acf = as.numeric(strsplit("-0.44 -0.05 -0.01 -0.03 0.12 -0.15 0.15 -0.04 -0.10 0.09 0.08 -0.07 0.06", " ")[[1]]),
             pacf = as.numeric(strsplit("-0.44 -0.31 -0.25 -0.25 -0.07 -0.21 -0.01 0.02 -0.09 -0.02 0.03 -0.02 0.01", " ")[[1]]))
data = list(data1, data2, data3)

for (i in 1:3) {
    if (i != 3) {
        data[[i]] = append(list(crit = 1.96*(cumsum(c(1, 2*data[[i]][[1]]^2))/100)^0.5), data[[i]])
    }

    else {
        data[[i]] = append(list(crit = 1.96*(cumsum(c(1, 2*data[[i]][[1]]^2))/99)^0.5), data[[i]])
    }
}

## plotting 3 plots
for (i in 1:3) {
    par(mfrow = c(1,2))
    plot(x = 1:13, y = data[[i]]$acf, type = "n", ylim = c(-1, 1), main = paste("data", i,"ACF"), xlab = "lag", ylab = "ACF")
    abline(h = 0)
    segments(x0 = 1:13, y0 = 0, x1 = 1:13, y1 = data[[i]]$acf)
    lines(x = 1:13, y = data[[i]]$crit[-14], lty = 2, col = "skyblue", lwd = 2)
    lines(x = 1:13, y = -data[[i]]$crit[-14], lty = 2, col = "skyblue", lwd = 2)

    plot(x = 1:13, y = data[[i]]$pacf, type = "n", ylim = c(-1, 1), main = paste("data", i,"PACF"), xlab = "lag", ylab = "PACF")
    abline(h = 0)
    segments(x0 = 1:13, y0 = 0, x1 = 1:13, y1 = data[[i]]$pacf)
    abline(h = 0.2, lty = 2, col = "skyblue", lwd = 2)
    abline(h = -0.2, lty = 2, col = "skyblue", lwd = 2)
}

### 2번 문제

In [ ]:
## data
z1 = scan("ex8_2a.txt")
z2 = scan("ex8_2b.txt")

##----------ex8_2a.txt data 분석----------
tsdisplay(z1, main = "ex8_2a.txt의 시계열 그림 및 SACF/SPACF", cex.main = 2)
t.test(z1)
adfTest(z1, lags = 1, type = "c") ## 평균 수준이 0이 아님

## AR(2) process : 선택 모형
fit1 = arima(z1, order = c(2,0,0), include.mean = T)
lmtest::coeftest(fit1)
t.test(fit1$residuals)
tseries::jarque.bera.test(fit1$residuals) ## H0 : normal distribution
portes::LjungBox(fit1, lags = c(6, 12, 18, 24)) ## H0 : rho1 = ... = rho_k(White noise)
astsa::sarima(z1, p=2, d=0, q=0)

## MA(3) process
fit2 = arima(z1, order = c(0,0,3), include.mean = T)
lmtest::coeftest(fit2) ## ma(3) parameter 유의함
astsa::sarima(z1, p=0, d=0, q=3)

## auto.arima 결과
forecast::auto.arima(z1, d=0, ic='aic', trace = T) ## 차분 없음

##----------ex8_2b.txt data 분석----------
tsdisplay(z2, main = "ex8_2b.txt의 시계열 그림 및 SACF/SPACF", cex.main = 2)
t.test(z2)
adfTest(z2, lags = 1, type = "c") ## 평균 수준이 0이 아님

## AR(1) process : 선택 모형
fit1 = arima(z2, order = c(1,0,0), include.mean = T)
lmtest::coeftest(fit1)
t.test(fit1$residuals)
tseries::jarque.bera.test(fit1$fit1$residuals)
portes::LjungBox(fit1, lags = c(6, 12, 18, 24))
astsa::sarima(z2, p=1, d=0, q=0)

## MA(3) process
fit2 = arima(z2, order = c(0,0,3), include.mean = T)
lmtest::coeftest(fit2)
astsa::sarima(z2, p=0, d=0, q=3)

## auto.arima 결과
forecast::auto.arima(z2, d=0, ic='aic', trace = T) ## 차분 없음

### 4번 문제

In [ ]:
start = c(44.00, 46.154)

for (i in 1:100) {
    start = c(start, 16.5 + 1.38*start[i+1] - 0.7*start[i])
}

plot(101:202, start, type = 'l', lwd = 2, col = "skyblue",
     xlab = "t", ylab = "forecasting", main = "시점 t = 100에서의 예측값", cex.main = 2)

start = c(44.38, 47.08)

for (i in 1:100) {
    start = c(start, 16.4864 + 1.38*start[i+1] - 0.7*start[i])
}

plot(101:202, start, type = 'l', lwd = 2, col = "skyblue",
     xlab = "t", ylab = "forecasting", main = "시점 t = 100에서의 예측값", cex.main = 2)

### 5번 문제

In [ ]:
## 최근 자료
recent_data = c(52.65, 54.87, 53.37, 48.21, 44.38, 47.08)
fit1 = arima(recent_data, order = c(2,0,0), fixed = c(1.38, -0.7, 51.52), include.mean = TRUE)

## 이후 25시차 예측
fore_fit1 <- forecast::forecast(fit1, 25)
plot(fore_fit1, xaxt = "n")
legend("topleft", c("원시계열", "예측값", "80% CI", "95% CI"), lty = c(1,1,NA,NA), pch = c(NA,NA,15,15), col = c(1, "blue", "lightblue", "grey"), lwd = c(2,2))
axis(1, at = seq(1, 31, by = 5), labels = seq(96, 126, by = 5))

### 6번 문제

`-` `ex10_4a.txt`

In [ ]:
## data
z1 = scan("ex10_4a.txt")

##----------(1, 2) 시계열 그림 및 ACF/PACF----------
tsdisplay(z1, main = "ex10_4a.txt의 시계열 그림", cex.main = 2)

##----------(3) 변환 및 차분----------
adfTest(z1, lags = 1, type = "ct") ## 귀무가설 기각, 확률적 추세 없음

## 추세를 이용한 분해
decmp1 = lm(z1~t1)
trend = predict(decmp1)
decomposed_z1 = decmp1$residuals
tsdisplay(decomposed_z1, lag.max = 25, main = "1차 추세모형 적합 후 잔차", cex.main = 2) ## 시각화

## 계절추세모형
decomposed_ts = ts(decomposed_z1, frequency = 5)
season_fit = lm(decomposed_ts~0+as.factor(cycle(decomposed_ts)))
tsdisplay(season_fit$residuals, main = "추세모형으로 추세 및 계절추세 분해", cex.main = 2) ## 체계적 성분 잔류

## 계절차분(주기 5)
lag5_decomposed_z1 = diff(decomposed_z1, lag = 5)
tsdisplay(lag5_decomposed_z1, main = "추세가 분해된 시계열에 주기가 5인 계절차분", cex.main = 2)

## 계절차분만
lag5_z1 = diff(z1, lag = 5)
tsdisplay(lag5_z1, main = "주기가 5인 계절차분만 진행", cex.main = 2)

##----------(4) 모형 적합----------
fit1 = arima(lag5_z1, order = c(0,0,1), include.mean = T)
lmtest::coeftest(fit1) ## 모수 유의성
t.test(fit1$residuals) ## 오차항의 평균 검정
tseries::jarque.bera.test(fit1$residuals) ## H0 : normal distribution
portes::LjungBox(fit1, lags = c(6, 12, 18, 24)) ## H0 : rho1 = ... = rho_k(White noise)

## 후보 모형
arima(lag5_decomposed_z1, order = c(0,0,1), include.mean = F)

##----------(5) 시뮬레이션 비교----------
z1_sim = sim_sarima(n = 10000, n.start = 5, model = list(ma = -0.8704, siorder=1, nseasons=5, sigma2 = 101.4**0.5), xintercept = 50.0582)
par(mfrow = c(2,2))
acf(z1, main = "원 시계열 SACF")
pacf(z1, main = "원 시계열 SPACF")
acf(z1_sim, main = "시뮬레이션 SACF", lag = 20)
pacf(z1_sim, main = "시뮬레이션 SPACF", lag = 20)

`-` `ex10_4b.txt`

In [ ]:
## data
z2 = scan("ex10_4b.txt")

##----------(1, 2) 시계열 그림 및 ACF/PACF----------
tsdisplay(z2, main = "ex10_4b.txt의 시계열 그림", cex.main = 2)

##----------(3) 변환 및 차분----------
t2 = 1:length(z2)
lmtest::bptest(lm(z2~t2)) ## 등분산성 만족, 변환 필요 없음
lag4_z2 = diff(z2, lag = 4)
tsdisplay(lag4_z2, main = "계절주기가 4인 계절차분", cex.main = 2)

## 단위근 검정
adfTest(lag4_z2, lag = 1, type = "nc") ## 단위근 없음

##----------(4) 모형 적합----------
fit2 = arima(lag4_z2, order = c(2, 0, 0), include.mean = F)
lmtest::coeftest(fit2) ## 모수 유의성
t.test(fit2$residuals) ## 오차항의 평균 검정
tseries::jarque.bera.test(fit2$residuals) ## H0 : normal distribution
portes::LjungBox(fit2, lags = c(6, 12, 18, 24)) ## H0 : rho1 = ... = rho_k(White noise)

##----------(5) 시뮬레이션 비교----------
z2_sim = sim_sarima(n = 10000, n.start = 4, model = list(ar = c(0.3157, 0.4597), siorder=1, nseasons=4, sigma2 = 7.03**0.5))
par(mfrow = c(2,2))
acf(z2, main = "원 시계열 SACF")
pacf(z2, main = "원 시계열 SPACF")
acf(z2_sim, main = "시뮬레이션 SACF", lag = 20)
pacf(z2_sim, main = "시뮬레이션 SPACF", lag = 20)

### 7번 문제

In [ ]:
## data
z = fma::hsales

##----------(1) 시계열 그림----------
plot.ts(z, main = "단독 주택 월별 거래량", cex.main = 2, ylab = "sales/month")

##----------(2) 변수 변환----------
t = 1:length(z)
lmtest::bptest(lm(z~t)) ## 등분산성 가정 기각
tseries::jarque.bera.test(lm(z~t)$residual) ## 정규성 만족...;;
tsdisplay(z, main = "원 시계열", cex.main = 2)
lmtest::bptest(lm(BoxCox(z, lambda = BoxCox.lambda(z))~t)) ## 등분산성 가정 기각
tsdisplay(BoxCox(z, lambda = BoxCox.lambda(z)),
          main = expression(paste("Box-Cox 변환(", lambda, " = 0.1455) 후 시계열")), cex.main = 2)

##----------(3) 정상화----------
adfTest(z, lags = 1, type = "c") ## 기각
lag12_z = diff(z, 12)
tsdisplay(lag12_z, main = "계절주기가 12인 계절차분", cex.main = 2)
adfTest(lag12_z, lags = 1, type = "nc") ## 기각

##----------(5) 모형 적합----------
fit1 = arima(z, order = c(1,0,0), seasonal = list(order = c(0,1,0), period=12))
fit2 = arima(z, order = c(1,0,0), seasonal = list(order = c(1,1,0), period=12)) ## 최종 선택 모형
fit3 = arima(z, order = c(1,0,0), seasonal = list(order = c(2,1,0), period = 12)) ## 얘가 제일 성능 좋음
lmtest::coeftest(fit1)
lmtest::coeftest(fit2)
lmtest::coeftest(fit3)

##----------(6) 모형 선택----------
forecast::auto.arima(z, test = "adf", seasonal = TRUE, trace = T, allowdrift = F, approximation = F) ## best : ARIMA(1,0,0)(2,1,0)

##----------(7) 잔차 검정----------
t.test(fit2$residuals) ## 오차항의 평균 검정
tseries::jarque.bera.test(fit2$residuals) ## H0 : normal distribution
portes::LjungBox(fit2, lags = c(6, 12, 18, 24)) ## H0 : rho1 = ... = rho_k(White noise) -> 24시차에서 기각
astsa::sarima(z, p=1, d=0, q=0, P = 1, D = 1, Q = 0, S = 12, no.constant = T) ## 24 시차 이후 p-value 낮음

t.test(fit3$residuals) ## 오차항의 평균 검정
tseries::jarque.bera.test(fit3$residuals) ## H0 : normal distribution
portes::LjungBox(fit3, lags = c(6, 12, 18, 24, 36)) ## H0 : rho1 = ... = rho_k(White noise) -> 24시차에서 기각
astsa::sarima(z, p=1, d=0, q=0, P = 2, D = 1, Q = 0, S = 12, no.constant = T) ## 35 시차 이후 p-value 낮음

fit4 = arima(z, order = c(1,0,0), seasonal = list(order = c(3,1,0), period = 12))
lmtest::coeftest(fit4)
fit5 = arima(z, order = c(1,0,0), seasonal = list(order = c(4,1,0), period = 12))
lmtest::coeftest(fit5)
fit6 = arima(z, order = c(1,0,0), seasonal = list(order = c(5,1,0), period = 12))
lmtest::coeftest(fit6)
# 성능 개선, 계수 유의성 지속

## ARIMA(1,0,0)(0,1,1) 모형 적합 및 잔차검정
fit_ma = arima(z, order = c(1,0,0), seasonal = list(order = c(0,1,1), period = 12))
fit_ma
lmtest::coeftest(fit_ma)
t.test(fit_ma$residuals) ## 오차항의 평균 검정
tseries::jarque.bera.test(fit_ma$residuals) ## H0 : normal distribution
portes::LjungBox(fit_ma, lags = c(6, 12, 18, 24, 36)) ## H0 : rho1 = ... = rho_k(White noise) -> 24시차에서 기각
astsa::sarima(z, p=1, d=0, q=0, P = 0, D = 1, Q = 1, S = 12, no.constant = T) ## 35 시차 이후 p-value 낮음

##----------(8) forecasting----------
forecast::forecast(fit_ma, 24)$mean |> round(2)
plot(forecast::forecast(fit_ma, 24), main = "차후 2년 간 주택의 월별 거래량 예측",
     cex.main = 2, xlab = "Year", ylab = "Sales/month")
legend("topleft", c("원시계열", "예측값", "80% CI", "95% CI"),
       lty = c(1,1,NA,NA), pch = c(NA,NA,15,15), col = c(1, "blue", "lightblue", "grey"), lwd = c(2,2))

### 8번 문제

In [ ]:
## data
z = expsmooth::ukcars

##----------(1) 시계열 그림----------
plot.ts(z, main = "승용차 분기별 생산 대수", cex.main = 2, ylab = "production/quarter", xlab = "Year")
points(z, pch = 16)

##----------(2) 변수 변환----------
t = 1:length(z)
lmtest::bptest(lm(z~t)) ## 등분산성 가정 기각
tseries::jarque.bera.test(lm(z~t)$residual) ## 정규성 만족...;;
tsdisplay(z, main = "원 시계열", cex.main = 2)
lmtest::bptest(lm(BoxCox(z, lambda = BoxCox.lambda(z))~t)) ## 등분산성 가정 기각
tsdisplay(BoxCox(z, lambda = BoxCox.lambda(z)),
          main = expression(paste("Box-Cox 변환(", lambda, " = 0.1455) 후 시계열")), cex.main = 2)
## 변수 변환 또 못함...

##----------(3) train_test_split----------
train = head(z, 105)
test = tail(z, 8)

##----------(4) 이동평균 모형 적합----------
mse_valid = 1:12
for (i in 1:12) {
    mse_valid[i] = mean((train[(i+1):length(train)] - SMA(train, i)[i:(length(train)-1)])^2)
}
which.min(mse_valid) ## 4. 시차와 연관 또한 있으므로 윈도우 크기를 4로 함

forecasting_SMA = ts(rep(SMA(train, 4)[length(train)], 8), start = c(2003,2), frequency = 4)

forecasting = c(SMA(train, 4)[length(train)]) ## 한 번에 뿌린 값
for (i in 1:7) {
    forecasting = c(forecasting, (3*forecasting[i] + test[i])/4)
}
forecasting_lag1_SMA = ts(forecasting, start = c(2003, 2),frequency = 4) ## sequential하게 뽑은 1시차 후 예측값

##----------(5) 지수평활 모형 적합----------
fit_hw = HoltWinters(train, seasonal = "additive") ## 등분산이므로 가법 모형
forecasting_HW = predict(fit_hw, 8) ## 한번에 다 뿌림

forecasting = c(predict(fit_hw, 1))

for (i in 1:7) {
    forecasting = c(forecasting, predict(HoltWinters(ts(c(train, test[1:i]), start = c(1977, 1), frequency = 4), seasonal = "additive"), 1))
}

forecasting_lag1_HW = ts(forecasting, start = c(2003, 2),frequency = 4) ## sequential하게 뽑은 1시차 후 예측값

##----------(6) 계절형 ARIMA 모형 적합----------
## 단위근 검정
adfTest(train, lags = 1, type = "c") ## 기각 못함

## 계절차분
lag4_train = diff(train, 4)
t.test(lag4_train)
tsdisplay(lag4_train, main = "계절주기가 4인 계절차분", cex.main = 2, lag.max = 36)
adfTest(lag4_train, lags = 1, type = "nc") ## 기각

## 모형 적합
fit1 = arima(train, order = c(1,0,0), seasonal = list(order = c(0,1,0), period=4))
fit2 = arima(train, order = c(1,0,0), seasonal = list(order = c(1,1,0), period=4))
fit3 = arima(train, order = c(1,0,0), seasonal = list(order = c(2,1,0), period=4))
lmtest::coeftest(fit1)
lmtest::coeftest(fit2)
lmtest::coeftest(fit3)

## 과대차분
fit4 = arima(train, order = c(1,0,0), seasonal = list(order = c(1,1,1), period=4))
fit5 = arima(train, order = c(1,0,1), seasonal = list(order = c(1,1,0), period=4))
fit6 = arima(train, order = c(1,0,0), seasonal = list(order = c(3,1,0), period=4))
fit7 = arima(train, order = c(1,0,0), seasonal = list(order = c(4,1,0), period=4)) ## 성능 감소
lmtest::coeftest(fit4)
lmtest::coeftest(fit5) ## ma(1) 파라미터 유의성 깨짐
lmtest::coeftest(fit6) ## sar(3) 파라미터 유의성 깨짐
lmtest::coeftest(fit7) ## sar(3), sar(4) 파라미터 유의성 깨짐

## auto.arima 결과
forecast::auto.arima(train, test = "adf", seasonal = TRUE, trace = T, allowdrift = F, approximation = F) ## best : ARIMA(1,0,1)(1,1,2)
fit8 = arima(train, order = c(1,0,1), seasonal = list(order = c(1,1,1), period=4)) ## 최고 성능
lmtest::coeftest(fit8) ## 모든 파라미터 유의

## 최종 모형 잔차검정
t.test(fit8$residuals) ## 오차항의 평균 검정
tseries::jarque.bera.test(fit8$residuals) ## H0 : normal distribution
portes::LjungBox(fit8, lags = c(4, 8, 12, 16))

## 예측값
forecasting_ARIMA = predict(fit8, 8)$pred ## 한번에 뿌림

forecasting = predict(fit8, 1)$pred

for (i in 1:7) {
    updated_model = arima(ts(c(train, test[1:i]), start = c(1977, 1), frequency = 4), order = c(1,0,1), seasonal = list(order = c(1,1,1), period = 4))
    forecasting = c(forecasting, predict(updated_model, 1)$pred)
}

forecasting_lag1_ARIMA = ts(forecasting, start = c(2003, 2),frequency = 4) ## sequential하게 뽑은 1시차 후 예측값

##----------(7) 모형 간 비교----------
forecasting_lagk = list(forecasting_SMA, forecasting_HW, forecasting_ARIMA)
forecasting_lag1 = list(forecasting_lag1_SMA, forecasting_lag1_HW, forecasting_lag1_ARIMA)

labls = c("SMA", "HW", "ARIMA")

for (i in 1:3) {
    print(paste(labls[i], "의 MSE = ", mean((test - forecasting_lagk[[i]])^2)))
    print(paste(labls[i], "의 MAE = ", mean(abs(test - forecasting_lagk[[i]]))))
    print(paste(labls[i], "의 MAPE = ", mean(abs(test - forecasting_lagk[[i]])/test)))
}

for (i in 1:3) {
    print(paste(labls[i], "의 MSE = ", mean((test - forecasting_lag1[[i]])^2)))
    print(paste(labls[i], "의 MAE = ", mean(abs(test - forecasting_lag1[[i]]))))
    print(paste(labls[i], "의 MAPE = ", mean(abs(test - forecasting_lag1[[i]])/test)))
}